In [ ]:
from pathlib import Path

In [ ]:
import iris
from wildfires.analysis import cube_plotting
from wildfires.data import DATA_DIR, HYDE, WWLLN, Datasets

### Load a cube containing the reference grid

In [ ]:
def ensure_bounds(cube):
    for coord in ("latitude", "longitude"):
        if not cube.coord(coord).has_bounds():
            cube.coord(coord).guess_bounds()

In [ ]:
ref_cube = iris.load_cube(
    "/gws/nopw/j04/jules/data/Ancillary/n96e/GL7/fixed/CRU-NCEPv7.landfrac.nc"
)
ensure_bounds(ref_cube)

#### Regridding scheme

In [ ]:
scheme = iris.analysis.AreaWeighted(mdtol=1)

#### Use mean state for popd.

In [ ]:
hyde = Datasets(HYDE()).select_variables("popd").dataset.get_mean_dataset()
ensure_bounds(hyde.cube)
hyde_reg_cube = hyde.cube.regrid(ref_cube, scheme)
# Add single time coordinate.
hyde_reg_cube_time = iris.cube.Cube(
    hyde_reg_cube.data[None],
    dim_coords_and_dims=[
        (
            iris.coords.DimCoord(
                [0], units="days since 2000-01-01", standard_name="time"
            ),
            0,
        ),
        (hyde_reg_cube.coord("latitude"), 1),
        (hyde_reg_cube.coord("longitude"), 2),
    ],
)
hyde_reg_cube_time.metadata = hyde_reg_cube.metadata
hyde_reg_cube_time

#### Use climatology for lightning.

In [ ]:
wwlln = WWLLN()
ensure_bounds(wwlln.cube)
wwlln_clim = wwlln.get_climatology_dataset(wwlln.min_time, wwlln.max_time)
wwlln_reg_cube = wwlln_clim.cube.regrid(ref_cube, scheme)
# Ensure the temporal coordinate is called 'time' and not 'month_number'.
wwlln_reg_cube.remove_coord("time")
temporal_coord = wwlln_reg_cube.coord("month_number")
temporal_coord.standard_name = "time"
temporal_coord.long_name = None
temporal_coord.var_name = None
wwlln_reg_cube

#### Write the modified files to disk

In [ ]:
target_dir = Path(DATA_DIR) / "n96e_data"
target_dir.mkdir(exist_ok=True)

In [ ]:
iris.save(hyde_reg_cube_time, str(target_dir / "hyde_mean_n96e.nc"))

In [ ]:
iris.save(wwlln_reg_cube, str(target_dir / "wwlln_clim_n96e.nc"))

In [ ]:
fig = cube_plotting(hyde_reg_cube, fig=plt.figure(figsize=(6.2, 3), dpi=130))

In [ ]:
fig = cube_plotting(wwlln_reg_cube, fig=plt.figure(figsize=(6.2, 3), dpi=130))

In [ ]:
hyde_reg_cube.data.mask.sum(), wwlln_reg_cube.data.mask.sum()